## Numerical Integration Methods

This notebook explores various **numerical integration techniques**.

Let $ f(x) $ be a continuous function defined on an interval $[a, b]$. The goal of numerical integration is to approximate the definite integral:
$$
\int_a^b f(x)\, dx
$$
The methods implemented and visualized include:

- **Rectangular Rule (Midpoint Method)**: Approximates the integral using rectangles. Height is based on the midpoint of each subinterval.
- **Trapezoidal Rule**: Approximates the region under the curve using trapezoids between sampled points.
- **Simpson's Rule**: Fits parabolas (quadratic polynomials) to small sections of the function and integrates them.
- **Adaptive Quadrature**: Recursively subdivides intervals based on estimated error to achieve high accuracy.
- **Gauss-Legendre Quadrature**: Uses optimal nodes and weights derived from Legendre polynomials to evaluate the integral with minimal evaluations.
- **Gauss-Laguerre Quadrature**: Suitable for semi-infinite integrals of the form $\int_0^\infty e^{-x}f(x)\,dx$.
- **Gauss-Hermite Quadrature**: Suitable for integrals over the whole real line with Gaussian weight, $\int_{-\infty}^{\infty} e^{-x^2}f(x)\,dx$.

Run the below cell first to import libraries.

In [ ]:
# Cell 1: Basic Functions and Imports, TO BE RUN FIRST EVERYTIME!
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.integrate import quad
from numpy.polynomial.legendre import leggauss, Legendre
from numpy.polynomial.laguerre import laggauss
from numpy.polynomial.hermite import hermgauss
import ipywidgets as widgets
from ipywidgets import interact, interactive, VBox, HBox, Layout
import seaborn as sns
sns.set_theme(style="whitegrid")
from IPython.core.display import display, HTML
display(HTML("<style>.output_subarea { max-width: 95% !important; }</style>"))



def parse_function(expr_str):
    try:
        f = lambda x: eval(expr_str, {"np": np, "x": x})
        _ = f(0.5)
        return f
    except Exception as e:
        print("Error parsing function:", e)
        return lambda x: np.zeros_like(x)


def rectangular_rule(f, a, b, n):
    dx = (b - a) / n
    midpoints = np.linspace(a + dx/2, b - dx/2, n)
    return np.sum(f(midpoints)) * dx

def trapezoidal_rule(f, a, b, n):
    x = np.linspace(a, b, n)
    y = f(x)
    dx = (b - a) / (n - 1)
    return (dx/2) * (y[0] + 2*np.sum(y[1:-1]) + y[-1])

def simpsons_rule(f, a, b, n):
    if n % 2 == 0:
        n += 1
    x = np.linspace(a, b, n)
    y = f(x)
    dx = (b - a) / (n - 1)
    S = y[0] + y[-1] + 4*np.sum(y[1:-1:2]) + 2*np.sum(y[2:-2:2])
    return S * dx / 3

def gauss_legendre_integration(f, a, b, n):
    std_nodes, std_weights = leggauss(n)
    t_nodes = 0.5*(b - a)*std_nodes + 0.5*(a + b)
    t_weights = 0.5*(b - a)*std_weights
    approx = np.sum(t_weights * f(t_nodes))
    return approx, t_nodes, t_weights, std_nodes, std_weights

def gauss_laguerre_integration(f, n):
    nodes, weights = laggauss(n)
    approx = np.sum(weights * f(nodes))
    return approx, nodes, weights

def gauss_hermite_integration(f, n):
    nodes, weights = hermgauss(n)
    approx = np.sum(weights * f(nodes))
    return approx, nodes, weights


def adaptive_integration(f, a, b):
    result, err = quad(f, a, b)
    return result, err


Use the interactive widgets provided to explore each method:

- **Method**: Select an integration method from the dropdown.
- **f(x)**: Type any valid Python expression for the function $ f(x) $. For example: `np.exp(-x**2)*np.sin(5*x)`.
- **Nodes n**: Controls the number of subintervals or quadrature nodes.
- **Lower Limit / Upper Limit**: Set the bounds $ a $ and $ b $ of integration.

The plot dynamically updates based on the choices:
- It shows the **true curve**, **approximated regions**, and **annotated geometric elements** (like rectangles, trapezoids, or nodes).
- It shows the **numerical result**, compared against a reference computed using `scipy.integrate.quad`.



In [ ]:
# Cell 3: Visualize Integration Methods

def visualize_integration_methods(method, expr, n, a, b):
    f = parse_function(expr)
    f_desc = expr

    if method in ["Rectangular", "Trapezoidal", "Simpson", "Gauss-Legendre", "Adaptive (quad)"]:
        x_vals = np.linspace(a, b, 1000)
        y_vals = f(x_vals)
        ref, ref_err = quad(f, a, b)

    if method == "Rectangular":
        approx = rectangular_rule(f, a, b, n)
        print("Rectangular Rule:")
        print("  • Divides [a,b] into equal subintervals and approximates the area using rectangles.")
        print("  • Each rectangle's height is determined by the function value at the midpoint.\n")
        plt.figure(figsize=(10,6), dpi=120)
        plt.plot(x_vals, y_vals, 'k-', lw=2, label=f"{f_desc}")
        plt.fill_between(x_vals, y_vals, color='lightgray', alpha=0.4, label="True Area")
        dx = (b - a) / n
        midpoints = np.linspace(a + dx/2, b - dx/2, n)
        for x0 in midpoints:
            rect = plt.Rectangle((x0 - dx/2, 0), dx, f(x0), color='skyblue', alpha=0.6)
            plt.gca().add_patch(rect)
            plt.annotate(f"mid={x0:.2f}\nw={dx:.2f}",
                         xy=(x0, f(x0)),
                         xytext=(x0, f(x0)+0.15),
                         arrowprops=dict(arrowstyle="->", color="navy"),
                         fontsize=9, color="navy", ha='center',
                         bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="navy", alpha=0.8))
        plt.title(f"Rectangular Rule on [{a}, {b}]\nApprox = {approx:.6f} (Ref: {ref:.6f} ± {ref_err:.2e})", fontsize=14)
        plt.xlabel("x", fontsize=12)
        plt.ylabel("f(x)", fontsize=12)
        plt.legend(fontsize=10)
        plt.tight_layout()
        plt.show()


    elif method == "Trapezoidal":
        approx = trapezoidal_rule(f, a, b, n)
        print("Trapezoidal Rule:")
        print("  • Approximates the area using trapezoids between successive points.")
        plt.figure(figsize=(10,6), dpi=120)
        plt.plot(x_vals, y_vals, 'k-', lw=2, label=f"{f_desc}")
        plt.fill_between(x_vals, y_vals, color='lightgray', alpha=0.4, label="True Area")
        x_trap = np.linspace(a, b, n)
        y_trap = f(x_trap)
        for i in range(n-1):
            xs = [x_trap[i], x_trap[i+1], x_trap[i+1], x_trap[i]]
            ys = [0, 0, y_trap[i+1], y_trap[i]]
            plt.gca().add_patch(plt.Polygon(np.column_stack((xs, ys)), color='peachpuff', alpha=0.7))
            plt.annotate(f"≈ {0.5*(x_trap[i+1]-x_trap[i])*(y_trap[i]+y_trap[i+1]):.2f}",
                         xy=((x_trap[i]+x_trap[i+1])/2, 0.5*(y_trap[i]+y_trap[i+1])),
                         fontsize=9, color="darkorange", ha='center',
                         bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="darkorange", alpha=0.8))
        plt.title(f"Trapezoidal Rule on [{a}, {b}]\nApprox = {approx:.6f} (Ref: {ref:.6f} ± {ref_err:.2e})", fontsize=14)
        plt.xlabel("x", fontsize=12)
        plt.ylabel("f(x)", fontsize=12)
        plt.legend(fontsize=10)
        plt.tight_layout()
        plt.show()


    elif method == "Simpson":
        approx = simpsons_rule(f, a, b, n)
        print("Simpson's Rule:")
        print("  • Uses parabolic (quadratic) fits over pairs of subintervals for higher accuracy.")
        plt.figure(figsize=(10,6), dpi=120)
        plt.plot(x_vals, y_vals, 'k-', lw=2, label=f"{f_desc}")
        plt.fill_between(x_vals, y_vals, color='lightgray', alpha=0.4, label="True Area")
        if n % 2 == 0:
            n += 1  # Ensure odd number for Simpson's rule
        x_simp = np.linspace(a, b, n)
        y_simp = f(x_simp)
        for i in range(0, n-2, 2):
            xs = x_simp[i:i+3]
            ys = y_simp[i:i+3]
            coeffs = np.polyfit(xs, ys, 2)
            poly = np.poly1d(coeffs)
            x_fit = np.linspace(xs[0], xs[-1], 50)
            plt.fill_between(x_fit, poly(x_fit), color='lightgreen', alpha=0.7)
            plt.annotate("Quadratic fit",
                         xy=((xs[0]+xs[-1])/2, poly((xs[0]+xs[-1])/2)),
                         xytext=((xs[0]+xs[-1])/2, poly((xs[0]+xs[-1])/2)+0.25),
                         arrowprops=dict(arrowstyle="->", color="forestgreen"),
                         fontsize=9, color="forestgreen", ha='center',
                         bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="forestgreen", alpha=0.8))
        plt.title(f"Simpson's Rule on [{a}, {b}]\nApprox = {approx:.6f} (Ref: {ref:.6f} ± {ref_err:.2e})", fontsize=14)
        plt.xlabel("x", fontsize=12)
        plt.ylabel("f(x)", fontsize=12)
        plt.legend(fontsize=10)
        plt.tight_layout()
        plt.show()


    elif method == "Adaptive (quad)":
        approx, err = adaptive_integration(f, a, b)
        print("Adaptive Quadrature Insight:")
        print("  • Automatically subdivides the interval where needed to meet the error tolerance.")
        plt.figure(figsize=(10,6), dpi=120)
        plt.plot(x_vals, y_vals, 'k-', lw=2, label=f"{f_desc}")
        plt.fill_between(x_vals, y_vals, color='lavender', alpha=0.4, label="Adaptive Integrated Area")
        plt.title(f"Adaptive Quadrature on [{a}, {b}]\nResult = {approx:.6f} ± {err:.2e}", fontsize=14)
        plt.xlabel("x", fontsize=12)
        plt.ylabel("f(x)", fontsize=12)
        plt.legend(fontsize=10)
        plt.tight_layout()
        plt.show()


    elif method == "Gauss-Legendre":
        approx, t_nodes, t_weights, std_nodes, std_weights = gauss_legendre_integration(f, a, b, n)
        print("Gauss-Legendre Quadrature Insight:")
        print("  • Nodes (red markers) represent the transformed zeros of the Legendre polynomial in [a,b].")
        print("  • Marker sizes are proportional to their weights, showing each node’s contribution.")
        print("  • The right plot displays the Legendre polynomial and its zeros (green dots).")
        print("  • This method is exact for all polynomials up to degree 2n-1.\n")
        ref, ref_err = quad(f, a, b)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,6), dpi=120)
        ax1.plot(x_vals, y_vals, 'k-', lw=2, label=f"{f_desc}")
        ax1.fill_between(x_vals, y_vals, color='lightgray', alpha=0.4, label="True Area")
        for xi, wi in zip(t_nodes, t_weights):
            ax1.axvline(xi, color='navy', linestyle='--', alpha=0.7)
            ax1.scatter(xi, f(xi), color='crimson', s=200*wi/np.max(t_weights), zorder=5)
            ax1.annotate("Node", xy=(xi, f(xi)),
                         xytext=(xi, f(xi)+0.15),
                         arrowprops=dict(arrowstyle="->", color="darkred"),
                         fontsize=9, color="darkred", ha='center',
                         bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="darkred", alpha=0.8))
        ax1.set_title("Gauss-Legendre Integration", fontsize=14)
        ax1.set_xlabel("x", fontsize=12)
        ax1.set_ylabel("f(x)", fontsize=12)
        ax1.legend(fontsize=10)
        ax1.grid(True)
        from numpy.polynomial.legendre import Legendre
        Pn = Legendre.basis(n)
        x_std = np.linspace(-1, 1, 400)
        y_std = Pn(x_std)
        ax2.plot(x_std, y_std, 'm-', lw=2, label=f"P_{n}(x)")
        ax2.scatter(std_nodes, np.zeros_like(std_nodes), s=100, color='green', label="Zeros (Nodes)")
        for z in std_nodes:
            ax2.axvline(z, color='green', linestyle='--', alpha=0.6)
            ax2.annotate("Zero", xy=(z, 0), xytext=(z, 0.08),
                         arrowprops=dict(arrowstyle="->", color="green"),
                         fontsize=8, color="green", ha='center',
                         bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="green", alpha=0.8))
        ax2.set_title(f"Legendre Polynomial P_{n}(x)", fontsize=14)
        ax2.set_xlabel("x (standard domain)", fontsize=12)
        ax2.set_ylabel("P_n(x)", fontsize=12)
        ax2.legend(fontsize=10)
        ax2.grid(True)
        plt.suptitle("Gauss-Legendre Quadrature Visualization", fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.93])
        plt.show()


    elif method == "Gauss-Laguerre":
        approx, nodes, weights = gauss_laguerre_integration(f, n)
        print("Gauss-Laguerre Quadrature:")
        print("  • For integrals of the form ∫₀∞ e^(-x)f(x)dx\n")
        ref, ref_err = quad(lambda x: np.exp(-x)*f(x), 0, np.inf)
        x_vals_gl = np.linspace(0, np.max(nodes)*1.5, 1000)
        y_vals_gl = np.exp(-x_vals_gl)*f(x_vals_gl)
        plt.figure(figsize=(10,6), dpi=120)
        plt.plot(x_vals_gl, y_vals_gl, 'k-', lw=2, label="e^(-x)*f(x)")
        plt.fill_between(x_vals_gl, y_vals_gl, color='lightblue', alpha=0.4, label="True Area")
        for xi, wi in zip(nodes, weights):
            plt.axvline(xi, color='saddlebrown', linestyle='--', alpha=0.7)
            plt.scatter(xi, np.exp(-xi)*f(xi), color='saddlebrown', s=200*wi/np.max(weights))
            plt.annotate("Node", xy=(xi, np.exp(-xi)*f(xi)),
                         xytext=(xi, np.exp(-xi)*f(xi)+0.15),
                         arrowprops=dict(arrowstyle="->", color="brown"),
                         fontsize=9, color="brown", ha='center',
                         bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="brown", alpha=0.8))
        plt.title(f"Gauss-Laguerre Quadrature (∫₀∞ e^(-x)f(x)dx), n={n}\nApprox = {approx:.6f} (Ref: {ref:.6f} ± {ref_err:.2e})", fontsize=14)
        plt.xlabel("x", fontsize=12)
        plt.ylabel("e^(-x)*f(x)", fontsize=12)
        plt.legend(fontsize=10)
        plt.tight_layout()
        plt.show()


    elif method == "Gauss-Hermite":
        approx, nodes, weights = gauss_hermite_integration(f, n)
        print("Gauss-Hermite Quadrature:")
        print("  • Used for integrals over (-∞,∞) with weight e^(-x²); nodes are zeros of the Hermite polynomial.")
        ref, ref_err = quad(lambda x: np.exp(-x**2)*f(x), -6, 6)
        x_vals_gh = np.linspace(np.min(nodes)*1.2, np.max(nodes)*1.2, 1000)
        y_vals_gh = np.exp(-x_vals_gh**2)*f(x_vals_gh)
        plt.figure(figsize=(10,6), dpi=120)
        plt.plot(x_vals_gh, y_vals_gh, 'k-', lw=2, label="e^(-x²)*f(x)")
        plt.fill_between(x_vals_gh, y_vals_gh, color='thistle', alpha=0.4, label="True Area")
        for xi, wi in zip(nodes, weights):
            plt.axvline(xi, color='mediumvioletred', linestyle='--', alpha=0.7)
            plt.scatter(xi, np.exp(-xi**2)*f(xi), color='mediumvioletred', s=200*wi/np.max(weights))
            plt.annotate("Node", xy=(xi, np.exp(-xi**2)*f(xi)),
                         xytext=(xi, np.exp(-xi**2)*f(xi)+0.15),
                         arrowprops=dict(arrowstyle="->", color="purple"),
                         fontsize=9, color="purple", ha='center',
                         bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="purple", alpha=0.8))
        plt.title(f"Gauss-Hermite Quadrature (∫₋∞∞ e^(-x²)f(x)dx), n={n}\nApprox = {approx:.6f} (Ref: {ref:.6f} ± {ref_err:.2e})", fontsize=14)
        plt.xlabel("x", fontsize=12)
        plt.ylabel("e^(-x²)*f(x)", fontsize=12)
        plt.legend(fontsize=10)
        plt.tight_layout()
        plt.show()

controls_visual = widgets.interactive(
    visualize_integration_methods,
    method = widgets.Dropdown(options=["Rectangular", "Trapezoidal", "Simpson", "Adaptive (quad)",
                                         "Gauss-Legendre", "Gauss-Laguerre", "Gauss-Hermite"],
                              value="Gauss-Legendre",
                              description="Method:"),
    expr = widgets.Text(value="np.exp(-x**2)*np.sin(5*x)",
                        description="f(x):",
                        layout=widgets.Layout(width='400px')),
    n = widgets.IntSlider(min=2, max=12, step=1, value=5, description="Nodes n:"),
    a = widgets.FloatSlider(min=0, max=1, step=0.1, value=0, description="Lower Limit:"),
    b = widgets.FloatSlider(min=1, max=5, step=0.1, value=np.pi, description="Upper Limit:")
)

display(VBox([widgets.HTML("<h2>Visual Exploration of Integration Methods</h2>"
                           "<p>Visualize classical methods and Gauss quadratures.</p>"),
              controls_visual]))


## Comparing Integration Methods

This section compares classical numerical integration techniques for the given function.

Let $ f(x) $ be a function defined on the interval $[a, b]$. The aim is to compute:
$
\int_a^b f(x) \, dx
$
using **numerical approximation** methods, such as,

- **Rectangular Rule (Midpoint)**:
  Uses rectangles with heights sampled at the midpoint of each subinterval:
  $
  \int_a^b f(x) dx \approx \sum_{i=1}^n f\left(x_i^*\right) \Delta x
  $
  where $ x_i^* $ is the midpoint of the $ i $-th subinterval.

- **Trapezoidal Rule**:
  Approximates the region using trapezoids between successive sample points:
  $
  \int_a^b f(x) dx \approx \frac{\Delta x}{2} \sum_{i=1}^{n} \left( f(x_{i-1}) + f(x_i) \right)
  $

- **Simpson’s Rule**:
  Approximates $ f(x) $ locally with quadratic polynomials over pairs of intervals:
  $
  \int_a^b f(x)\, dx \approx \frac{\Delta x}{3} \left[ f(x_0) + 4f(x_1) + 2f(x_2) + \dots + f(x_n) \right]
  $
  Requires an **odd number of nodes** (even number of subintervals).

- **Gauss-Legendre Quadrature**:
  Uses roots of Legendre polynomials as nodes, for integrating polynomials up to degree $2n-1$ exactly:
  $
  \int_{a}^{b} f(x)\,dx \approx \sum_{i=1}^{n} w_i f(x_i)
  $
  Nodes and weights are determined from the Legendre polynomial of degree $n$.

- **Adaptive Quadrature (`scipy.integrate.quad`)**:
  Uses recursive subdivision and local error estimation to adaptively sample $ f(x) $. This is used here as a **reference** method.

---


- **f(x)**: Enter a Python expression for the function to integrate, e.g. `np.exp(-x**2)*np.sin(5*x)`.
- **Nodes/Subint n**: Controls the number of intervals.
- **Lower Limit / Upper Limit**: Specify bounds of integration.

After inputting a function and adjusting parameters, the output generates:

1. **Rectangular Rule Visualization**
2. **Trapezoidal Rule Visualization**
3. **Simpson’s Rule Visualization**
4. **Gauss-Legendre Quadrature**
5. **Reference Adaptive Quadrature (`quad`)**
6. **Summary** with:
   - Approximated values: The computed integral using each method.
   - Absolute error ($ |I_{\text{approx}} - I_{\text{ref}}| $) and relative errors

The shaded geometric regions in each subplot visually demonstrate how each method approximates the integral.


In [ ]:
# Cell 2: Compare Integration Methods and Visualize

def compare_methods(expr, n, a, b):
    f = parse_function(expr)
    f_desc = expr

    ref, ref_err = adaptive_integration(f, a, b)

    approx_rect = rectangular_rule(f, a, b, n)
    approx_trap = trapezoidal_rule(f, a, b, n)
    approx_simp = simpsons_rule(f, a, b, n)
    approx_gauss, gauss_nodes, gauss_weights, std_nodes, std_weights = gauss_legendre_integration(f, a, b, n)

    x_vals = np.linspace(a, b, 1000)
    y_vals = f(x_vals)

    fig, axs = plt.subplots(2, 3, figsize=(18,10))
    axs = axs.flatten()

    # Rectangular Rule
    dx = (b - a) / n
    midpoints = np.linspace(a + dx/2, b - dx/2, n)
    axs[0].plot(x_vals, y_vals, 'k-', lw=2, label=f"{f_desc}")
    axs[0].fill_between(x_vals, y_vals, color='lightgray', alpha=0.3)
    for m in midpoints:
        rect = plt.Rectangle((m - dx/2, 0), dx, f(m), color='cyan', alpha=0.5)
        axs[0].add_patch(rect)
    axs[0].set_title(f"Rectangular Rule\nApprox = {approx_rect:.6f}")
    axs[0].set_xlabel("x"); axs[0].set_ylabel("f(x)")
    axs[0].grid(True)

    # Trapezoidal Rule
    x_trap = np.linspace(a, b, n)
    y_trap = f(x_trap)
    axs[1].plot(x_vals, y_vals, 'k-', lw=2, label=f"{f_desc}")
    axs[1].fill_between(x_vals, y_vals, color='lightgray', alpha=0.3)
    for i in range(n-1):
        xs = [x_trap[i], x_trap[i+1], x_trap[i+1], x_trap[i]]
        ys = [0, 0, y_trap[i+1], y_trap[i]]
        axs[1].add_patch(plt.Polygon(np.column_stack((xs, ys)), color='orange', alpha=0.5))
    axs[1].set_title(f"Trapezoidal Rule\nApprox = {approx_trap:.6f}")
    axs[1].set_xlabel("x"); axs[1].grid(True)

    #Simpson's Rule
    if n % 2 == 0:
        n_used = n + 1
    else:
        n_used = n
    x_simp = np.linspace(a, b, n_used)
    y_simp = f(x_simp)
    axs[2].plot(x_vals, y_vals, 'k-', lw=2, label=f"{f_desc}")
    axs[2].fill_between(x_vals, y_vals, color='lightgray', alpha=0.3)
    for i in range(0, n_used-2, 2):
        xs = x_simp[i:i+3]
        ys = y_simp[i:i+3]
        coeffs = np.polyfit(xs, ys, 2)
        poly = np.poly1d(coeffs)
        x_fit = np.linspace(xs[0], xs[-1], 50)
        axs[2].fill_between(x_fit, poly(x_fit), color='green', alpha=0.5)
    axs[2].set_title(f"Simpson's Rule\nApprox = {approx_simp:.6f}")
    axs[2].set_xlabel("x"); axs[2].grid(True)

    # Gauss-Legendre Quadrature
    axs[3].plot(x_vals, y_vals, 'k-', lw=2, label=f"{f_desc}")
    axs[3].fill_between(x_vals, y_vals, color='lightgray', alpha=0.3)
    std_nodes, _ = leggauss(n)
    for xi, wi in zip(std_nodes, gauss_weights):
        x_trans = 0.5*(b - a)*xi + 0.5*(a + b)
        axs[3].axvline(x_trans, color='blue', linestyle='--', alpha=0.7)
        axs[3].scatter(x_trans, f(x_trans), color='red', s=200*wi/np.max(gauss_weights), zorder=5)
    axs[3].set_title(f"Gauss-Legendre Quadrature\nApprox = {approx_gauss:.6f}")
    axs[3].set_xlabel("x"); axs[3].grid(True)

    # -- Adaptive Quadrature (Reference) --
    axs[4].plot(x_vals, y_vals, 'k-', lw=2, label=f"{f_desc}")
    axs[4].fill_between(x_vals, y_vals, color='lightgray', alpha=0.3)
    axs[4].set_title(f"Adaptive Quadrature (quad)\nRef = {ref:.6f} ± {ref_err:.2e}")
    axs[4].set_xlabel("x"); axs[4].grid(True)


    axs[5].axis('off')
    summary_text = (
        f"{'Method':<20}{'Approximation':>15}{'Abs Error':>15}{'Rel Error (%)':>20}\n"
        f"{'-'*70}\n"
        f"{'Rectangular':<20}{approx_rect:15.6f}{abs(approx_rect-ref):15.6f}{100*abs(approx_rect-ref)/abs(ref):20.2f}\n"
        f"{'Trapezoidal':<20}{approx_trap:15.6f}{abs(approx_trap-ref):15.6f}{100*abs(approx_trap-ref)/abs(ref):20.2f}\n"
        f"{'Simpson':<20}{approx_simp:15.6f}{abs(approx_simp-ref):15.6f}{100*abs(approx_simp-ref)/abs(ref):20.2f}\n"
        f"{'Gauss-Legendre':<20}{approx_gauss:15.6f}{abs(approx_gauss-ref):15.6f}{100*abs(approx_gauss-ref)/abs(ref):20.2f}\n"
        f"{'Adaptive (quad)':<20}{ref:15.6f}{0.0:15.6f}{0.00:20.2f}\n\n"
        "Observation: Gauss-Legendre quadrature, by optimally choosing nodes (zeros of the Legendre polynomial)\n"
        "and computing corresponding weights, often yields the smallest error compared to classical methods."
    )
    axs[5].text(0, 0.5, summary_text, fontsize=12, fontfamily='monospace')

    plt.suptitle("Comparison of Integration Methods", fontsize=18)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

    #print(summary_text)

controls_compare = widgets.interactive(
    compare_methods,
    expr = widgets.Text(value="np.exp(-x**2)*np.sin(5*x)",
                        description="f(x):",
                        layout=Layout(width='400px')),
    n = widgets.IntSlider(min=2, max=12, step=1, value=5, description="Nodes/Subint n:"),
    a = widgets.FloatSlider(min=0, max=1, step=0.1, value=0, description="Lower Limit:"),
    b = widgets.FloatSlider(min=1, max=5, step=0.1, value=np.pi, description="Upper Limit:")
)

display(VBox([widgets.HTML("<h2>Integration Methods Comparison</h2>"
                           "<p>Enter a function and adjust the parameters. "
                           "The plots compare classical methods (Rectangular, Trapezoidal, Simpson) with Gauss-Legendre quadrature "),
              controls_compare]))


## Adaptive Gauss–Legendre Quadrature

This section animates **adaptive quadrature method** using **Gauss–Legendre quadrature** as the base rule. The animation visually illustrates how the integration domain is recursively subdivided based on local error estimates, allowing for accurate numerical integration, especially in the presence of  function variation or oscillations.

#### Gauss–Legendre Quadrature

Gauss–Legendre quadrature is an accurate method for approximating definite integrals. It uses the roots of Legendre polynomials as nodes and is exact for polynomials of degree up to $2n-1$, where $n$ is the number of nodes. The quadrature rule on the interval $[-1, 1]$ is:

$$
\int_{-1}^{1} f(x) \, dx \approx \sum_{i=1}^{n} w_i f(x_i),
$$

where:
- $x_i$ are the nodes (roots of the Legendre polynomial $P_n(x)$),
- $w_i$ are the corresponding weights.

To evaluate the integral on a general interval $[a, b]$, we apply a change of variables:

$$
x = \frac{b-a}{2} \xi + \frac{a+b}{2}, \quad \xi \in [-1, 1],
$$

which transforms the integral to:

$$
\int_a^b f(x) \, dx = \frac{b-a}{2} \int_{-1}^1 f\left( \frac{b-a}{2} \xi + \frac{a+b}{2} \right) \, d\xi.
$$

#### Adaptive Quadrature

Fixed-node quadrature rules may fail to achieve desired accuracy for functions with localized irregularities. Adaptive quadrature addresses this by:

1. Approximating the integral over an interval $[a, b]$ using Gauss–Legendre quadrature:
   $$
   I_{\text{whole}} \approx \int_a^b f(x)\, dx.
   $$

2. Dividing the interval into two subintervals and computing:
   $$
   I_{\text{left}} \approx \int_a^{m} f(x)\, dx, \quad
   I_{\text{right}} \approx \int_m^b f(x)\, dx, \quad \text{where } m = \frac{a + b}{2}.
   $$

3. Estimating the local error by:
   $$
   \text{error} = \left| I_{\text{whole}} - (I_{\text{left}} + I_{\text{right}}) \right|.
   $$

4. If $\text{error} < \text{tol}$ (user-specified tolerance) or a maximum depth is reached, accept the result. Otherwise, recursively apply the same procedure to each subinterval.



In [ ]:
# Cell 4: Adaptive Gauss–Legendre Quadrature Animation
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
from IPython.display import HTML

def gauss_legendre_quadrature(f, a, b, degree=4):
    nodes, weights = np.polynomial.legendre.leggauss(degree)
    x_trans = 0.5 * (b - a) * nodes + 0.5 * (a + b)
    w_trans = 0.5 * (b - a) * weights
    return np.sum(w_trans * f(x_trans))

def adaptive_gauss_progress(f, a, b, tol, degree=4, depth=0, max_depth=10,
                            history=None, order_counter=None, parent_coord=None):
    if history is None:
        history = []
    if order_counter is None:
        order_counter = [0]

    current_mid = (a + b) / 2.0
    current_coord = (depth, current_mid)

    I_whole = gauss_legendre_quadrature(f, a, b, degree)
    mid = current_mid
    I_left = gauss_legendre_quadrature(f, a, mid, degree)
    I_right = gauss_legendre_quadrature(f, mid, b, degree)
    error = np.abs(I_whole - (I_left + I_right))

    order_counter[0] += 1
    order = order_counter[0]
    accepted = (error < tol or depth >= max_depth)
    int_val = (I_left + I_right) if accepted else None
    history.append((a, b, error, depth, order, accepted, int_val, parent_coord))

    if accepted:
        return I_left + I_right, history, order_counter
    else:
        left_val, history, order_counter = adaptive_gauss_progress(f, a, mid, tol/2, degree, depth+1, max_depth, history, order_counter, current_coord)
        right_val, history, order_counter = adaptive_gauss_progress(f, mid, b, tol/2, degree, depth+1, max_depth, history, order_counter, current_coord)
        return left_val + right_val, history, order_counter

def run_adaptive(f, a, b, tol, degree, max_depth):
    history = []
    order_counter = [0]
    result, history, _ = adaptive_gauss_progress(f, a, b, tol, degree, 0, max_depth, history, order_counter)
    exact = quad(f, a, b)[0]
    return result, history, exact

def animate_adaptive_quadrature(f, a, b, tol, degree, max_depth):
    result, history, exact = run_adaptive(f, a, b, tol, degree, max_depth)
    max_order = max(order for (_,_,_,_,order,_,_,_) in history)
    x_vals = np.linspace(a, b, 1000)

    fig = plt.figure(figsize=(16, 9), facecolor="white", tight_layout=True)
    gs = gridspec.GridSpec(2, 2, figure=fig,
                          width_ratios=[3, 1],
                          height_ratios=[2, 1],
                          hspace=0.4, wspace=0.3)

    ax_func = fig.add_subplot(gs[0, :])
    ax_error = fig.add_subplot(gs[1, 0])
    ax_info = fig.add_subplot(gs[1, 1])
    ax_info.axis('off')

    y_vals = f(x_vals)
    y_margin = 0.1*(np.max(y_vals) - np.min(y_vals))
    func_ylim = (np.min(y_vals)-y_margin, np.max(y_vals)+y_margin)

    all_errors = [err for (_,_,err,_,_,_,_,_) in history]
    error_ylim = (0.1*min(all_errors), 10*max(all_errors))

    cmap = mcolors.LinearSegmentedColormap.from_list("error_cmap", ["#2c7bb6", "#d7191c"])

    def update(frame):
        ax_func.cla(); ax_error.cla(); ax_info.cla()
        ax_info.axis('off')

        ax_func.plot(x_vals, y_vals, color="navy", lw=1.5, label="f(x)")
        ax_func.set_xlim(a, b)
        ax_func.set_ylim(func_ylim)
        ax_func.tick_params(axis='both', colors='black', labelsize=8)
        ax_func.set_title(f"Adaptive Subdivision Process (Step {frame}/{max_order})",
                         color='black', fontsize=12, pad=10)

        for (x0, x1, err, depth, order, accepted, int_val, _) in history:
            if order <= frame:
                norm_err = min(err/tol, 1.0)
                color = cmap(norm_err)
                ax_func.axvspan(x0, x1, color=color, alpha=0.6, ec="k", lw=0.3)
                if (x1 - x0) < (b - a)/15:
                    y_pos = func_ylim[0] + 0.85*(func_ylim[1]-func_ylim[0])
                    status = "✓" if accepted else "→"
                    ax_func.text((x0+x1)/2, y_pos, status,
                                fontsize=8, ha='center', va='center',
                                bbox=dict(facecolor='white', alpha=0.9,
                                          edgecolor='gray', boxstyle='round,pad=0.2'))

        orders = [order for (_,_,err,_,order,_,_,_) in history if order <= frame]
        errors = [err for (_,_,err,_,order,_,_,_) in history if order <= frame]
        if orders:
            ax_error.semilogy(orders, errors, 'o-', color='darkred',
                             markersize=4, lw=1, alpha=0.7)
            ax_error.set_xlim(0, max_order+1)
            ax_error.set_ylim(error_ylim)
            ax_error.axhline(tol, color='green', ls='--', lw=1, alpha=0.8, label='Tolerance')
            ax_error.tick_params(axis='both', colors='black', labelsize=8)
            ax_error.set_xlabel("Processing Order", fontsize=10, color='black')
            ax_error.set_ylabel("Error Estimate", fontsize=10, color='black')
            ax_error.legend(fontsize=8, loc='upper right')
            ax_error.set_title("Error Evolution", color='black', fontsize=12, pad=10)

        cum_int = sum(int_val for (_,_,_,_,_,accepted, int_val,_) in history if accepted and int_val is not None and order <= frame)
        relative_error = np.abs(cum_int - exact) / np.abs(exact) if exact != 0 else np.nan
        total_accepted = sum(1 for (_,_,_,_,_,accepted,_,_) in history if accepted)
        max_depth_used = max((depth for (_, _, _, depth, _, _, _, parent) in history if parent is not None), default=0)

        text_content = [
            f"Step: {frame}/{max_order}",
            f"Cumulative Integral: {cum_int:.6f}",
            f"Exact Value (quad): {exact:.6f}",
            f"Relative Error: {relative_error:.2e}",
            f"Current Tolerance: {tol:.1e}",
            f"Accepted Intervals: {total_accepted}",
            f"Max Depth Reached: {max_depth_used}",
            "",
            "Recent Actions:"
        ]

        recent = [h for h in history if h[4] == frame][-3:]
        for h in recent:
            action = "ACCEPTED" if h[5] else "SUBDIVIDED"
            text_content.append(f"• {action} interval [{h[0]:.2f}, {h[1]:.2f}] (err={h[2]:.1e})")

        ax_info.text(0.05, 0.95, "\n".join(text_content),
                    fontsize=10, color='black',
                    va='top', ha='left', linespacing=1.5,
                    transform=ax_info.transAxes,
                    bbox=dict(facecolor='#f7f7f7', ec='gray',
                             boxstyle='round,pad=0.5', alpha=0.9))

    ani = animation.FuncAnimation(fig, update, frames=range(1, max_order+1),
                                  interval=1000, repeat=False)
    plt.close()
    return ani, result, exact, history

def interactive_animation(function, tol, degree, max_depth, a, b):
    if function == "np.sin":
        f_callable = np.sin
    elif function == "np.exp":
        f_callable = np.exp
    else:
        f_callable = eval(function)

    ani, result, exact, history = animate_adaptive_quadrature(f_callable, a, b, tol, degree, max_depth)
    info = (
        f"Adaptive Gauss–Legendre Quadrature\n"
        f"Function: {function}\n"
        f"Interval: [{a:.2f}, {b:.2f}]\n"
        f"Tolerance: {tol:.1e}\n"
        f"Nodes: {degree}\n"
        f"Max Depth: {max_depth}\n\n"
        f"Adaptive Result: {result:.6f}\n"
        f"Exact (quad): {exact:.6f}\n"
        f"Total Accepted Intervals: {sum(1 for (_,_,_,_,_,accepted,_,_) in history if accepted)}"
    )
    print(info)
    html_video = ani.to_html5_video()
    responsive_html = f'''
    <div style="max-width: 900px; margin: auto;">
     <style>
      video {{ width: 100%; height: auto; display: block; max-height: 600px; }}
     </style>
      {html_video}
    </div>
    '''
    return HTML(responsive_html)

interact(interactive_animation,
         function=widgets.Dropdown(options=[("Sine (np.sin)", "np.sin"),
                                             ("Gaussian (exp(-x^2))", "lambda x: np.exp(-x**2)"),
                                             ("Exponential (np.exp)", "np.exp"),
                                             ("Oscillatory (sin(10x))", "lambda x: np.sin(10*x)")],
                                     value="lambda x: np.sin(10*x)",
                                     description="Function:"),
         tol=widgets.FloatLogSlider(value=1e-4, base=10, min=-6, max=-2, description="Tolerance:"),
         degree=widgets.IntSlider(value=4, min=2, max=10, step=1, description="Nodes:"),
         max_depth=widgets.IntSlider(value=8, min=3, max=15, step=1, description="Max Depth:"),
         a=widgets.FloatSlider(value=0, min=0, max=1, step=0.05, description="Lower Limit a:"),
         b=widgets.FloatSlider(value=3.14, min=0.5, max=10, step=0.1, description="Upper Limit b:")
         )


interactive(children=(Dropdown(description='Function:', index=3, options=(('Sine (np.sin)', 'np.sin'), ('Gauss…

<function __main__.interactive_animation(function, tol, degree, max_depth, a, b)>